In [27]:
import datetime as dt
from odds_and_other_projections import *

current_year = dt.date.today().year
capital_538 = 100000

In [28]:
# Getting odds and 538 data in one place
data = retrieve_538()
odds = retrieve_odds()
merged = pd.merge(data, odds, on = 'Home_Team')
merged = merged[['Date', 'Away_Team_x', 'Home_Team', 'Away_Prob_x', 'Home_Prob_x', 'Home_Odds', 'Away_Odds',
       'Home_Prob_y', 'Away_Prob_y']]
merged.columns = ['Date', 'Away_Team', 'Home_Team', 'Away_Prob_538', 'Home_Prob_538', 'Home_Odds', 'Away_Odds',
       'Home_Prob_Odds', 'Away_Prob_Odds']

# Formatting data
merged['Home_Prob_538'] = merged.Home_Prob_538.str.strip('%')
merged['Away_Prob_538'] = merged.Away_Prob_538.str.strip('%')
merged['Home_Prob_538'] = merged.Home_Prob_538.astype('float') / 100.0
merged['Away_Prob_538'] = merged.Away_Prob_538.astype('float') / 100.0
merged['Home_Prob_Odds'] = merged.Home_Prob_Odds / 100.0
merged['Away_Prob_Odds'] = merged.Away_Prob_Odds / 100.0
current_day = dt.date.today().day
merged['Date_Day'] = merged.Date.dt.day
merged = merged[merged.Date_Day == current_day]
merged.drop_duplicates(inplace = True, subset = ['Home_Team', 'Away_Team'])
merged.drop(['Date_Day'], axis = 1, inplace = True)

# Defining KC calculator function
def kc(row, kelly, Home):
    if Home:
        diff = row.Home_Prob_538 - row.Home_Prob_Odds
        if diff<0:
            return 0
        else:
            p = row.Home_Prob_538
            q = 1-p
            ml = row.Home_Odds
            if ml>=0:
                b = (ml/100)
            if ml<0:
                b = (100/abs(ml))
            kc = ((p*b) - q) / b
            if (kc > 0.5) & (kc<0.6):
                return kc/(kelly+2)
            if (kc > 0.6) & (kc<0.7):
                return kc/(kelly+4)
            if kc > 0.7:
                return kc/(kelly+7)
            else:
                return kc/kelly
    if not Home:
        diff = row.Away_Prob_538 - row.Away_Prob_Odds
        if diff<0:
            return 0
        else:
            p = row.Away_Prob_538
            q = 1-p
            ml = row.Away_Odds
            if ml>=0:
                b = (ml/100)
            if ml<0:
                b = (100/abs(ml))
            kc = ((p*b) - q) / b
            if (kc > 0.5) & (kc<0.6):
                return kc/(kelly+2)
            if (kc > 0.6) & (kc<0.7):
                return kc/(kelly+4)
            if kc > 0.7:
                return kc/(kelly+7)
            else:
                return kc/kelly
    
# Creating bets
merged['Home_KC_538'] = merged.apply(kc, axis = 1, kelly = 10, Home = True)
merged['Away_KC_538'] = merged.apply(kc, axis = 1, kelly = 10, Home = False)
merged['Bet_538'] = merged.apply(lambda x: capital_538 * x.Home_KC_538 if x.Home_KC_538>0
        else capital_538 * x.Away_KC_538, axis = 1)

,Date,Away_Team,Home_Team,Away_Prob_538,Home_Prob_538,Home_Odds,Away_Odds,Home_Prob_Odds,Away_Prob_Odds,Home_KC_538,Away_KC_538,Bet_538
0,1900-04-24,Marlins,Braves,0.39,0.61,-134.0,114.0,0.572650,0.467290,0.008740,0.000000,874.000000
4,1900-04-24,Giants,Nationals,0.59,0.41,162.0,-194.0,0.381679,0.659864,0.004580,0.000000,458.024691
8,1900-04-24,Blue Jays,Astros,0.46,0.54,-134.0,104.0,0.572650,0.490196,0.000000,0.000000,0.000000
9,1900-04-24,White Sox,Twins,0.54,0.46,-106.0,-110.0,0.514563,0.523810,0.000000,0.003400,340.000000
13,1900-04-24,Pirates,Cubs,0.42,0.58,-162.0,136.0,0.618321,0.423729,0.000000,0.000000,0.000000
14,1900-04-24,Orioles,Angels,0.39,0.61,-210.0,176.0,0.677419,0.362319,0.000000,0.004341,434.090909
19,1900-04-24,Rangers,Athletics,0.48,0.52,100.0,-118.0,0.500000,0.541284,0.004000,0.000000,400.000000
22,1900-04-24,Mets,Diamondbacks,0.53,0.47,156.0,-186.0,0.390625,0.650350,0.013026,0.000000,1302.564103
26,1900-04-24,Dodgers,Padres,0.55,0.45,142.0,-168.0,0.413223,0.626866,0.006268,0.000000,626.760563
27,1900-04-24,Royals,Mariners,0.36,0.64,-200.0,168.0,0.666667,0.373134,0.000000,0.000000,0.000000
